#Função a ser aproximada: Griewank

## **Plot 2D da função Griewank**

![alt text](http://2.bp.blogspot.com/_otsWqKrA2RI/TJHH5DguZjI/AAAAAAAAACo/01VNZLSvrlo/s400/fn_griewank.bmp)

### **Definição e ótimo global:**

![alt text](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/images/griewankLatex.png)

#Declarar a função

In [ ]:
import math
def Griewank(input):
    sum = 0
    for x in input:
        sum += x * x
    product = 1
    for i in range(len(input)):
        product *= math.cos(input[i] / math.sqrt(i + 1))
    return 1 + sum / 4000 - product

In [ ]:
#Teste de chamada para duas variáveis
Griewank([0,0])

#Amostragem da função a ser aproximada

In [ ]:
import random
# Seed para reprodução de resultados
seed = 10
random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

n = 2
samples = 10_000
lower_bound = -512
upper_bound = 512

random_samples = []
for sample in range(samples):
  input = []
  for i in range(n):
    input.append(random.uniform(lower_bound, upper_bound))
  random_samples.append(input)

#converte para numpy array
import numpy as np
random_samples = np.array(random_samples)

random_samples[0:5] # 5 primeiras amostras

In [ ]:
random_samples.shape

In [ ]:
#Saída para cada input
y = []
for i in random_samples:
  y.append(Griewank(i))

#Separar base em treino e validação

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(random_samples, y, test_size = 0.2, shuffle=True, random_state=seed)

X_train = np.asarray(X_train)
X_val = np.asarray(X_val)
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)

#Treinamento da Rede Neural para aproximar a função

In [ ]:
# Importando as bibliotecas do Keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
# Iniciando a Rede Neural
classifier = Sequential()

# Adicionando a camada de input e a primeira camada escondida
classifier.add(Dense(units = 80, activation = 'tanh', input_dim = n))
classifier.add(Dropout(rate = 0.1))

classifier.add(Dense(units = 50, activation = 'tanh'))
classifier.add(Dropout(rate = 0.1))

# Adicionando a camada de saída
classifier.add(Dense(units = 1, activation = 'linear'))

# Compilando a rede neural
classifier.compile(optimizer = 'adam', loss = 'mse')

# Visualização das camadas da rede
classifier.summary()

In [ ]:
# Treinando a rede neural
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(filepath='best.h5', save_best_only=True)
history = classifier.fit(X_train, y_train, batch_size = 64, epochs = 300, validation_data = (X_val,y_val), callbacks=[mc])

#Avaliar erros de treino e validação

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='treino')
plt.plot(history.history['val_loss'], label='validação')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

#Fazer previsões e avaliar a rede treinada

###Exercício 1: Avalie a rede treinada com 500 dados amostrados com distribuição uniforme. Salve em uma variável chamada 'y_test' os dados gerados.

###Exercício 2: Utilize a rede neural (best.h5) para prever o resultado da função para as entradas geradas acima. Chame a variável com os resultados de 'y_pred'.

In [ ]:
#Real vs. Previsto
plt.figure(figsize=(20,5))
plt.plot(y_pred, label='previsto')
plt.plot(y_test, label='real')
plt.title('Aproximação Grienwank')
plt.ylabel('y')
plt.xlabel('index')
plt.legend()
plt.show()

###Exercício 3: Faça um plot como o acima, mas utilizando somente 50 amostras da base de teste.

In [ ]:
#Real vs. Previsto
plt.scatter(x=y_pred, y=y_test)
plt.title('Aproximação Grienwank')
plt.ylabel('Previsto')
plt.xlabel('Real')

x = [0, 110]
plt.plot(x, x, color='red'); #reta onde y=x
plt.show()

In [ ]:
# calcular o erro para a base de teste
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score

mse = mean_squared_error(y_pred, y_test)
print('MSE: ',mse)
r2 = r2_score(y_pred, y_test)
print('R2: ', r2)